In [1]:
%%capture
%run 02_transaction.ipynb

In [2]:
from IPython.display import Image
from random import randint as ri

Create random txs.

In [3]:
def rtxs(n): return [TX(rh(),rh(),ri(1,9),ri(1,9)/10,0) for _ in range(n)] 
print(txs2str(rtxs(2)))

fr:            📗 0x713df0f942...508
hash:          📑 0x6b42b9970c...f18
to:            💟 0x398140370a...ea6
value:         8.0 eth
fee:           0.4 eth
nonce:         0
time:          Sat Apr  3 18:08:46 2021
signed:        False

fr:            🕈 0xe23cff01e1...0c5
hash:          📄 0x5e4813241b...4e6
to:            🔵 0xcf7939c7c3...fb3
value:         2.0 eth
fee:           0.9 eth
nonce:         0
time:          Sat Apr  3 18:08:46 2021
signed:        False



## Merkle Tree

A binary tree in which every leaf node is a hash of a specific piece of data, in our case a tx. 

In a system like Bitcoin or Ethereum the merkle root is included in the block header, not the list of txs itself. This makes it very efficient to test if a specific tx is inclued in the tree.

For more in depth [here](https://www.youtube.com/watch?v=3giNelTfeAk).

<div>
<img src="https://changelly.com/blog/wp-content/uploads/2020/01/Merkle-Tree-1.png" width="500"/>
</div>


`MerkleTree` is a wrapper around the `mt` dictionary which stores the hashes for each height in the tree.

In [4]:
class MerkleTree:
    def __init__(self, txs):
        self.mt = {}
        leafs = [tx.hash for tx in txs]
        if len(leafs)%2!=0: leafs.append(sha('0x0'))
        self.mt['1'] = leafs
        self.merkle(leafs)
        
    def merkle(self, leafs):
        parents  = []
        while len(leafs) > 1:
            for i in range(0, len(leafs), 2):
                l = leafs[i]
                if i+1>=len(leafs): r = '0x0'
                else              : r = leafs[i+1]
                parents.append(sha(l+r))
            leafs = parents
            self.mt[f'{len(self)+1}'] = parents
            parents = []
    
    @property
    def root(self)      : return self.mt[str(len(self))][0]
    def __eq__ (self, o): return self.root == o.root
    def __len__(self)   : return len(self.mt)
    def __getitem__(self, k): return self.mt[str(k)]
    def __str__(self):
        s = ''
        for k,v in self.mt.items(): 
            s += f'height {k}'
            for h in v: s += f'\n {ph(h)}'
            s += '\n'
        return s

Create Merkle Tree from `N` random txs.

In [5]:
N   = 8
txs = rtxs(N)

mt = MerkleTree(txs); print(mt)

height 1
 💩 0x4396e89fb2...c55
 📌 0x66ff039daf...b97
 💍 0x279efbfe85...818
 🔪 0xc483749b21...163
 🕎 0xe85a8febae...d20
 💢 0x3c081c2e2b...523
 📳 0x8dd0ff431f...3a4
 💩 0x43e73a1f62...4d2
height 2
 📈 0x62dfc45282...3ef
 🕏 0xe9b65e0b40...6ec
 👸 0x12feff919f...6d1
 📅 0x5ff42917f4...ec7
height 3
 🔛 0xb5bbdce4ee...ba7
 🔄 0x9e078582f1...d7c
height 4
 💰 0x4ad4619814...69e



In [6]:
assert len(mt) == 4

The merkle tree root hash is the top most node. If anything in the tree changes the root changes as well.

In [7]:
ph(mt.root)

'💰 0x4ad4619814...69e'

If we change something in any tx the whole Merkle Tree changes. Here we change the `value` of the third tx. Compare the hashes below with the ones above.

In [8]:
txs[2].value = 500

changed_mt = MerkleTree(txs); print(changed_mt)

height 1
 💩 0x4396e89fb2...c55
 📌 0x66ff039daf...b97
 🕟 0xf9d575acd1...86a
 🔪 0xc483749b21...163
 🕎 0xe85a8febae...d20
 💢 0x3c081c2e2b...523
 📳 0x8dd0ff431f...3a4
 💩 0x43e73a1f62...4d2
height 2
 📈 0x62dfc45282...3ef
 🕋 0xe5550fd67e...e35
 👸 0x12feff919f...6d1
 📅 0x5ff42917f4...ec7
height 3
 💮 0x487d9b2721...89a
 🔄 0x9e078582f1...d7c
height 4
 💳 0x4dfde676ee...f07



In [9]:
assert mt != changed_mt

## Merkle Proof

Proof that `tx` at position `pos` in the merkle tree `mt` is part of it by using a corresponding merkle branch `mb`.

A merkle branch consists of those nodes that are needed to build up the root hash of the merkle tree. For example:

Merkle branch `[5,3,0, ..., x]` stands for the following: 
- node *5* from height *0*
- node *3* from height *1*
- node *0* from height *2*
- ...
- node *x* from height `len(mt)-1`

In [10]:
def proof(mt, mb, tx):
    if mb[0]%2 != 0: cn = sha(tx.hash     +mt[1][mb[0]])
    else           : cn = sha(mt[1][mb[0]]+tx.hash)
    for i in range(2, len(mt)):
        if mb[i-1]%2 != 0: cn = sha(cn            +mt[i][mb[i-1]])
        else             : cn = sha(mt[i][mb[i-1]]+cn)
    return cn == mt.root

Examples of some txs with their corresponding merkle branch.

In [11]:
tx = txs[0]
mb = [1,1,1]

assert proof(mt, mb, tx)

In [12]:
tx = txs[1]
mb = [0,1,1]

assert proof(mt, mb, tx)

In [13]:
tx = txs[4]
mb = [5,3,0]

assert proof(mt, mb, tx)